In [1]:
# Empiezo importando las librerias que usar en el projecto

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json

In [2]:
# creo un DataFrame en pandas(pd) a partir de los datos contenidos en m_details.csv.
pd.set_option('display.max_colwidth', None) # quiero ver los valores completos.
df = pd.read_csv("/Users/benjaminzelaya/Desktop/carpeta/Proyecto_ML_Individual/ETL - EDA/datasets reestructurados para empezar ETL/data Reestructurados/m_details.csv")

# Imprimo los primeros registros del DataFrame
df.head()

,release_date,budget,revenue,id
0,1995-10-30,30000000,373554033.0,862
1,1995-12-15,65000000,262797249.0,8844
2,1995-12-22,0,0.0,15602
3,1995-12-22,16000000,81452156.0,31357
4,1995-02-10,0,76578911.0,11862


## Los valores nulos de los campos **`revenue`**, **`budget`** deben ser rellenados por el número **`0`**.


In [3]:
# Verifico como son los valores de la columna Revenue

pd.set_option('display.max_colwidth', None)
df['revenue'].head()

0    373554033.0
1    262797249.0
2            0.0
3     81452156.0
4     76578911.0
Name: revenue, dtype: float64

In [4]:
# tipo de valor de la columna

df['revenue'].dtypes

dtype('float64')

In [5]:
# Rellenar los valores nulos con 0 en la columnas "revenue"
df["revenue"].fillna(0, inplace=True)

In [6]:
# Verifico como son los valores de la columna budget

pd.set_option('display.max_colwidth', None)
df['budget'].head()

0    30000000
1    65000000
2           0
3    16000000
4           0
Name: budget, dtype: object

In [7]:
# Rellenar los valores nulos con 0 en la columnas "budget"
df["budget"].fillna(0, inplace=True)

In [8]:
pd.set_option('display.max_colwidth', None)
df['budget']

0        30000000
1        65000000
2               0
3        16000000
4               0
           ...   
45461           0
45462           0
45463           0
45464           0
45465           0
Name: budget, Length: 45466, dtype: object

## Los valores nulos del campo **`release date`** deben eliminarse.

## De haber fechas, deberán tener el formato **`AAAA-mm-dd`**, además deberán crear la columna **`release_year`** donde extraerán el año de la fecha de estreno.

In [9]:
df['release_date'].isnull().mean()

0.001913517793516034

In [17]:
# Elimino los valones nulos de la columna release_date
df.dropna(subset=['release_date'], inplace=True)


In [14]:
# Verifico que los mismos fueron realmente eliminados de la columna
df['release_date'].isnull().mean()

0.0

In [15]:
# Ahora siguiendo con lo indicado convierto la columna "release_date" en tipo datetime, utilizando pandas.
df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce')

In [18]:
#  creo la columna "release_year" como se indica con el formato AAAA-mm-dd , para extraer los datos de estreno de las peliculas.
df['release_year'] = df['release_date'].dt.year
df['release_year'] = df['release_year'].astype(int)


# Si falla asegurarse que se hizo la eliminacion de los valores nulos corrextamente en la linea #65 

In [19]:
df['release_year'].head() # Verifico como quedo la columna con los años en enteros.

0    1995
1    1995
2    1995
3    1995
4    1995
Name: release_year, dtype: int64

In [20]:
columna_eliminada = ['release_date','id']
df.drop(columns=columna_eliminada, inplace=True)
# Elimino la columna release_date porque sus datos ya fueron extraidos en columnas diferentes y mas ordenados.


In [21]:
# verifico como quedo el dataframe con la columna agregada.
df.head()


,budget,revenue,release_year
0,30000000,373554033.0,1995
1,65000000,262797249.0,1995
2,0,0.0,1995
3,16000000,81452156.0,1995
4,0,76578911.0,1995


## Crear la columna con el retorno de inversión, llamada **`return`** con los campos **`revenue`** y **`budget`**, dividiendo estas dos últimas **`revenue / budget`**, cuando no hay datos disponibles para calcularlo, deberá tomar el valor **`0`**.


In [22]:
print(df['budget'].dtype)

object


In [23]:
print(df['revenue'].dtype)

float64


* algunos de los valores en la columna 'revenue' o 'budget' son de tipo 'str' (cadena de texto) en lugar de números. La división solo es compatible con valores numéricos, y al intentar dividir una cadena de texto ('str') con un número ('float'). asi que realizaremos la conversion correspondiente usando pandas  a traves de .to_numeric.

In [24]:
df['revenue'] = pd.to_numeric(df['revenue'], errors='coerce')
df['budget'] = pd.to_numeric(df['budget'], errors='coerce')

In [25]:
# Ahora si puedo crear la  columna 'return' con el retorno de inversión de las pelicuals haciendo la division correspondiente.
df['return'] = df.apply(lambda row: row['revenue'] / row['budget'] if (not pd.isnull(row['revenue']) and not pd.isnull(row['budget']) and row['budget'] != 0) else 0, axis=1)

# utilizando aplly para aplicar la funcion lambda en cada una de las columnas indicadas, con la  condición que verifica si  el valor de la columna 'revenue' como el de 'budget' no son nulos.
# y a su vez  si el valor de 'budget' es diferente de cero, para evitar division en 0 si es que faltan valores, y le indico que axis 1 para obtener los valores de cada una de las filas .

In [26]:
df['return'].head()

0    12.451801
1     4.043035
2     0.000000
3     5.090760
4     0.000000
Name: return, dtype: float64

In [27]:
df.head()

,budget,revenue,release_year,return
0,30000000,373554033.0,1995,12.451801
1,65000000,262797249.0,1995,4.043035
2,0,0.0,1995,0.000000
3,16000000,81452156.0,1995,5.090760
4,0,76578911.0,1995,0.000000


In [28]:
movie_data_1 = df

# Creo el dataframe con las columnas indicadas guardando el mismo dentro la carpeta ETL/EDA del reposotorio
movie_data_1.to_csv("m_details_modificado.csv", index=False)